In [17]:
# Import class files

import sys
import os
parent_dir = os.path.abspath(os.path.join(os.getcwd(), os.pardir))
sys.path.append(parent_dir)

In [18]:
from classes.grid import Grid
from queue import PriorityQueue
from tqdm import tqdm
import math

example = open('example.txt', 'r').read()
puzzle = open('puzzle.txt', 'r').read()

input = puzzle

# Part 1

In [19]:
class RaceGrid(Grid):
    def __init__(self, grid):
        super().__init__(grid)
        self.start_pos = self.get_tiles('S').pop()
        self.end_pos = self.get_tiles('E').pop()
        self.walls = self.get_tiles('#')
    
    def find_distances(self):
        '''
        Uses Dijkstra's to find all of the paths via cheating that save (exactly)
        'timesave' picoseconds from the baseline 'shortest_path_no_cheats'
        '''

        seen = set()

        best_cheats = 0

        distances = {}
        prev_node = {}

        # Nodes are (tile, tile that we cheated on, direction that we cheated)

        nodes_to_visit = PriorityQueue()

        init_tiles = self.get_tiles('.').union(self.get_tiles('E').union(self.get_tiles('S')))

        # Need to get all cheats possible - cheats uniquely identified by start and dir

        for tile in init_tiles:
            distances[(tile)] = math.inf
            prev_node[(tile)] = None
            nodes_to_visit.put((math.inf, (tile)))

        nodes_to_visit.put((0, (self.start_pos)))
        distances[(self.start_pos)] = 0
        
        #print(distances, prev_node)
        
        while nodes_to_visit.qsize() > 0:
            cur_node = nodes_to_visit.get()
            cur_node_cost = cur_node[0]
            cur_node_coords = cur_node[1]

            # Find neighbours
            neighbours = set()

            for dir in ['^','v','<','>']:
                adj_1 = self.get_relative(cur_node_coords[0],cur_node_coords[1], dir)
                if adj_1[2] in ['.','E'] and adj_1[0] != -1:
                    neighbours.add((adj_1[0], adj_1[1]))
            
            for neighbour in neighbours:
                neighbour_cheated = neighbour[1]

                # It costs 2 picoseconds when cheating instead of 1
                edge_cost = 1

                # Cost to get to new node
                neighbour_cost = cur_node_cost + edge_cost

                if neighbour_cost < distances[neighbour]:
                    prev_node[neighbour] = cur_node
                    distances[neighbour] = neighbour_cost
                    nodes_to_visit.put((neighbour_cost, neighbour))


        self.distances = distances
        return distances, prev_node, best_cheats
    
    def find_timesaves(self, min_timesave):
        '''
        Finds all of the ways we can get a timesave that has at least min_timesaves picoseconds from original
        '''
        num_timesaves = 0

        # Loop through passable tiles and check the available cheats and if there is a timesave
        for tile in self.get_tiles('S').union(self.get_tiles('.')):
            for dir in ['^','<','>','v']:
                one_step = self.get_relative(tile[0],tile[1],dir)
                two_steps = self.get_relative(tile[0],tile[1],dir,step=2)
                if one_step[2] == '#' and two_steps[2] in ['.','E'] and two_steps[0] != -1:
                    orig_dist = self.distances[(two_steps[0],two_steps[1])]
                    cheated_dist = self.distances[tile] + 2
                    if orig_dist-cheated_dist >= min_timesave and cheated_dist < orig_dist:
                        #print(tile, two_steps, orig_dist, cheated_dist)
                        num_timesaves += 1
        
        return num_timesaves


In [20]:
race_grid_map = input.split('\n')

race_grid = RaceGrid(race_grid_map)
#race_grid.print_grid()

# Use A* to get the shortest path without any cheating 
dij_info = race_grid.find_distances()
race_grid.find_timesaves(100)

1445

# Part 2

In [21]:
class RaceGrid(Grid):
    def __init__(self, grid):
        super().__init__(grid)
        self.start_pos = self.get_tiles('S').pop()
        self.end_pos = self.get_tiles('E').pop()
        self.walls = self.get_tiles('#')
    
    def find_distances(self):
        '''
        Uses Dijkstra's to find all of the paths via cheating that save (exactly)
        'timesave' picoseconds from the baseline 'shortest_path_no_cheats'
        '''

        seen = set()

        best_cheats = 0

        distances = {}
        prev_node = {}

        # Nodes are (tile, tile that we cheated on, direction that we cheated)

        nodes_to_visit = PriorityQueue()

        init_tiles = self.get_tiles('.').union(self.get_tiles('E').union(self.get_tiles('S')))

        # Need to get all cheats possible - cheats uniquely identified by start and dir

        for tile in init_tiles:
            distances[(tile)] = math.inf
            prev_node[(tile)] = None
            nodes_to_visit.put((math.inf, (tile)))

        nodes_to_visit.put((0, (self.start_pos)))
        distances[(self.start_pos)] = 0
        
        #print(distances, prev_node)
        
        while nodes_to_visit.qsize() > 0:
            cur_node = nodes_to_visit.get()
            cur_node_cost = cur_node[0]
            cur_node_coords = cur_node[1]

            # Find neighbours
            neighbours = set()

            for dir in ['^','v','<','>']:
                adj_1 = self.get_relative(cur_node_coords[0],cur_node_coords[1], dir)
                if adj_1[2] in ['.','E'] and adj_1[0] != -1:
                    neighbours.add((adj_1[0], adj_1[1]))
            
            for neighbour in neighbours:
                neighbour_cheated = neighbour[1]

                # It costs 2 picoseconds when cheating instead of 1
                edge_cost = 1

                # Cost to get to new node
                neighbour_cost = cur_node_cost + edge_cost

                if neighbour_cost < distances[neighbour]:
                    prev_node[neighbour] = cur_node
                    distances[neighbour] = neighbour_cost
                    nodes_to_visit.put((neighbour_cost, neighbour))


        self.distances = distances
        return distances, prev_node, best_cheats
    
    def find_timesaves(self, min_timesave):
        '''
        Finds all of the ways we can get a timesave that has at least min_timesaves picoseconds from original
        '''
        num_timesaves = 0

        # Loop through passable tiles and check the available cheats and if there is a timesave
        for tile in tqdm(self.get_tiles('S').union(self.get_tiles('.'))):
            for dest_tile in self.get_tiles('.').union(self.get_tiles('E')):
                dist = abs(tile[0] - dest_tile[0]) + abs(tile[1] - dest_tile[1])

                # Cheat too long or cheat is actually just following the path?
                if dist > 20 or self.distances[tile]+dist == self.distances[dest_tile]:
                    continue

                orig_dist = self.distances[dest_tile]
                cheated_dist = self.distances[tile]+dist

                if orig_dist-cheated_dist >= min_timesave and cheated_dist < orig_dist:
                    #print(tile, two_steps, orig_dist, cheated_dist)
                    num_timesaves += 1
        
        return num_timesaves


In [22]:
race_grid_map = input.split('\n')

race_grid = RaceGrid(race_grid_map)
#race_grid.print_grid()

# Use A* to get the shortest path without any cheating 
dij_info = race_grid.find_distances()
race_grid.find_timesaves(100)

100%|██████████| 9432/9432 [00:52<00:00, 178.70it/s]


1008040